In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from IPython.display import Image, display
import time

def scrape_nate_news():
    """
    네이트 뉴스의 6개 섹션(최신뉴스, 정치, 경제, 사회, 세계, IT/과학)에서 
    뉴스 제목, 링크, 이미지를 스크래핑하는 함수
    """
    base_url = "https://news.nate.com"
    
    # 각 섹션별 URL 매핑
    sections = {
        "최신뉴스": "/recent?mid=n0100",
        "정치": "/recent?mid=n0101", 
        "경제": "/recent?mid=n0102",
        "사회": "/recent?mid=n0103",
        "세계": "/recent?mid=n0104",
        "IT/과학": "/recent?mid=n0105"
    }
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    
    all_news = {}
    
    for section_name, section_path in sections.items():
        print(f"\n{'='*50}")
        print(f"📰 {section_name} 섹션")
        print(f"{'='*50}")
        
        try:
            # 각 섹션 페이지 요청
            url = base_url + section_path
            response = requests.get(url, headers=headers)
            response.raise_for_status()
            
            soup = BeautifulSoup(response.content, 'html.parser')
            
            # 뉴스 항목들 찾기 (네이트 뉴스 구조에 맞게 선택자 조정)
            news_items = soup.find_all('div', class_='mduSubjectList') or \
                        soup.find_all('li', class_='mduSubject') or \
                        soup.find_all('div', class_='mduSubject')
            
            if not news_items:
                # 다른 구조일 경우를 위한 대체 선택자
                news_items = soup.select('.mduSubject, .subjectList li, .newsList li')
            
            section_news = []
            
            for i, item in enumerate(news_items[:10]):  # 상위 10개 뉴스만
                try:
                    # 제목과 링크 추출
                    title_element = item.find('a') or item.find('span', class_='subject')
                    if not title_element:
                        continue
                        
                    title = title_element.get_text(strip=True)
                    link = title_element.get('href', '')
                    
                    # 상대 경로를 절대 경로로 변환
                    if link:
                        full_link = urljoin(base_url, link)
                    else:
                        full_link = ""
                    
                    # 이미지 추출
                    img_element = item.find('img')
                    image_url = ""
                    
                    if img_element:
                        img_src = img_element.get('src', '')
                        if img_src:
                            # 상대 경로를 절대 경로로 변환
                            image_url = urljoin(base_url, img_src)
                    
                    # 결과 저장
                    news_data = {
                        'title': title,
                        'link': full_link,
                        'image_url': image_url
                    }
                    
                    section_news.append(news_data)
                    
                    # 출력
                    print(f"\n[{i+1}] 제목: {title}")
                    print(f"    링크: {full_link}")
                    
                    if image_url:
                        print(f"    이미지: {image_url}")
                        try:
                            # 이미지 표시
                            display(Image(url=image_url, width=200))
                        except Exception as img_error:
                            print(f"    이미지 로드 실패: {img_error}")
                    else:
                        print("    이미지: 없음")
                    
                    print("-" * 30)
                    
                except Exception as e:
                    print(f"뉴스 항목 처리 중 오류: {e}")
                    continue
            
            all_news[section_name] = section_news
            
            # 요청 간격 조정 (서버 부하 방지)
            time.sleep(1)
            
        except Exception as e:
            print(f"{section_name} 섹션 스크래핑 중 오류 발생: {e}")
            all_news[section_name] = []
    
    return all_news

def display_news_summary(news_data):
    """스크래핑된 뉴스 데이터의 요약 출력"""
    print(f"\n{'='*60}")
    print("📊 뉴스 스크래핑 결과 요약")
    print(f"{'='*60}")
    
    for section, articles in news_data.items():
        print(f"{section}: {len(articles)}개 기사")
    
    total_articles = sum(len(articles) for articles in news_data.values())
    print(f"\n총 스크래핑된 기사 수: {total_articles}개")

# 실행 예제
if __name__ == "__main__":
    print("네이트 뉴스 스크래핑을 시작합니다...")
    
    # 뉴스 스크래핑 실행
    news_data = scrape_nate_news()
    
    # 결과 요약 출력
    display_news_summary(news_data)
    
    print("\n스크래핑이 완료되었습니다!")

네이트 뉴스 스크래핑을 시작합니다...

📰 최신뉴스 섹션

[1] 제목: 최민호 세종시장 'U대회기 인수' 유럽 출장…민주당 인사들 "이 와중에"최민호 세종시장. / 뉴스1 (세종=뉴스1) 장동열 기자 = 최민호 세종시장이 오는 24일 2027 충청권 하계세계대학경기대회 대회기 인수 등 공무를 위해 독일로 떠난다. 이를 ...
    링크: https://news.nate.com/view/20250723n10110?mid=n0100
    이미지: https://thumbnews.nateimg.co.kr/news90///news.nateimg.co.kr/orgImg/na/2025/07/23/7408174_high.jpg


------------------------------

[2] 제목: 미일 "日상호관세 15%" 타결…일본車 27.5% →15% 깜짝 합의(종합)지난 2월 백악관에서 정상회담을 갖고 있는 도널드 트럼프 미국 대통령(오른쪽)과 이시바 시게루 일본 총리. 2025.02.07. ⓒ 로이터=뉴스1 ⓒ News1 이지예 객원기자 (서울...
    링크: https://news.nate.com/view/20250723n09861?mid=n0100
    이미지: https://thumbnews.nateimg.co.kr/news90///news.nateimg.co.kr/orgImg/na/2025/07/23/7325680_high.jpg


------------------------------

[3] 제목: 트럭에 사랑의 선풍기 싣고(부산=뉴스1) 윤일지 기자 = 전국 대부분 지역에 폭염특보가 발효 중인 23일 오전 부산진구 부산시민공원 야외주차장에서 대한적십자사 부산지사 자원봉사원들이 취약계층...
    링크: https://news.nate.com/view/20250723n09405?mid=n0100
    이미지: https://thumbnews.nateimg.co.kr/news90///news.nateimg.co.kr/orgImg/na/2025/07/23/7408239_high.jpg


------------------------------

[4] 제목: 마무리의 공 하나는 팀을 웃고 울린다…정해영-한준수 배터리의 안이했던 볼 배합 하나가 대역전승을 그르쳤다야구에서 가장 중압감이 큰 보직을 꼽으라면 단연 마무리 투수다. 자신의 공 하나에 팀 동료들이 애써 만들어놓은 승리를 날려버릴 수 있기 때문이다. 그래서 마무리 투수...
    링크: https://news.nate.com/view/20250723n09930?mid=n0100
    이미지: https://thumbnews.nateimg.co.kr/news90///news.nateimg.co.kr/orgImg/sg/2025/07/23/20250723505545.jpg


------------------------------

[5] 제목: 도우인시스 코스닥시장 상장기념식(서울=뉴스1) = 23일 서울 영등포구 한국거래소에서 열린 도우인시스의 코스닥시장 상장기념식에서 유도석 한국IR협회 상무(왼쪽부터), 민경욱 한국거래소 코스닥시장본부...
    링크: https://news.nate.com/view/20250723n10106?mid=n0100
    이미지: https://thumbnews.nateimg.co.kr/news90///news.nateimg.co.kr/orgImg/na/2025/07/23/7408277_high.jpg


...

------------------------------

[6] 제목: SPC그룹 "AI 기술 적용…안전 스마트 공장 설립"SPC그룹이 안전 경영을 강화하기 위해 인공지능(AI)이 적용된 안전 스마트 공장을 세우겠다고 밝혔다.SPC는 그룹 쇄신을 위한 대표 협의체 ‘변화와 혁신 추진단’ 출범과 함께 첫 회의를 갖고 이 같은 내용을 의결...
    링크: https://news.nate.com/view/20250723n00327?mid=n0105
    이미지: 없음
------------------------------

[7] 제목: "AI로 사회문제 해결"…메가존클라우드, UNDP와 베트남서 협력(왼쪽부터)베트남 하노이 UN 그린원 사무소에서 메가존클라우드 염동훈 대표와 UNDP 라므라 하리디 상임대표가 업무협약 체결 후 기념촬영을 하고 있다.(사진=메가존클라우...
    링크: https://news.nate.com/view/20250723n09397?mid=n0105
    이미지: https://thumbnews.nateimg.co.kr/news90///news.nateimg.co.kr/orgImg/ns/2025/07/23/NISI20250723_0001900216_web.jpg


------------------------------

[8] 제목: 3500만원 들여 '젊어지는 수술' 받은 60대女…"두 가지 크게 후회" [헬스톡]안면거상술을 받은 페기 린 마르케티(60)가 수술 후 젊어진 모습 공개했다. 사진=SNS 캡처 [파이낸셜뉴스] 최근 멕시코에서 약 2만 5천 달러(약 3500만 원)를 들여 안면거...
    링크: https://news.nate.com/view/20250723n07321?mid=n0105
    이미지: https://thumbnews.nateimg.co.kr/news90///news.nateimg.co.kr/orgImg/fn/2025/07/23/202507230922386144_l.jpg


------------------------------

[9] 제목: 베스핀글로벌, 노르마와 양자 클라우드 확산 MOU노르마 정현철 대표(왼쪽)와 베스핀글로벌 허양호 한국 대표가 업무 협약 체결 후 기념촬영을 하고 있다. 베스핀글로벌 제공 [파이낸셜뉴스] 베스핀글로벌은 23일 양자 컴...
    링크: https://news.nate.com/view/20250723n09285?mid=n0105
    이미지: https://thumbnews.nateimg.co.kr/news90///news.nateimg.co.kr/orgImg/fn/2025/07/23/202507230844104966_l.jpg


------------------------------

[10] 제목: 90년생 백민경 교수, APEC서 혁신 연구 성과 인정…AI로 단백질 구조 해독[서울=뉴시스] 아스파이어(ASPIRE)상 최종 우승자로 선정된 백민경 서울대 교수. (사진=과기정통부 제공) 2025.07.23. photo@newsis.com *재판매 및 DB 금지 [서울=뉴시스]...
    링크: https://news.nate.com/view/20250723n07971?mid=n0105
    이미지: https://thumbnews.nateimg.co.kr/news90///news.nateimg.co.kr/orgImg/ns/2025/07/23/NISI20250723_0001900215_web.jpg


------------------------------

📊 뉴스 스크래핑 결과 요약
최신뉴스: 10개 기사
정치: 10개 기사
경제: 10개 기사
사회: 10개 기사
세계: 10개 기사
IT/과학: 10개 기사

총 스크래핑된 기사 수: 60개

스크래핑이 완료되었습니다!


In [2]:
import os
import requests
from bs4 import BeautifulSoup
import time
from urllib.parse import urljoin
import re

def download_one_episode(title, no, url):
    """
    네이버 웹툰의 특정 회차 이미지들을 다운로드하는 함수
    
    Args:
        title (str): 웹툰 제목
        no (int/str): 회차 번호
        url (str): 웹툰 회차 URL
    """
    
    # 디렉토리 생성
    base_dir = "img"
    webtoon_dir = os.path.join(base_dir, title, str(no))
    
    try:
        os.makedirs(webtoon_dir, exist_ok=True)
        print(f"디렉토리 생성: {webtoon_dir}")
    except Exception as e:
        print(f"디렉토리 생성 실패: {e}")
        return
    
    # 헤더 설정 (네이버 웹툰은 Referer 체크가 중요)
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Referer': 'https://comic.naver.com/'
    }
    
    try:
        print(f"웹툰 페이지 접근 중: {url}")
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # 웹툰 이미지들 찾기
        # 네이버 웹툰의 이미지는 주로 .wt_viewer 클래스 내부에 있음
        viewer_area = soup.find('div', class_='wt_viewer')
        
        if not viewer_area:
            # 대체 선택자 시도
            viewer_area = soup.find('div', {'id': 'comic_view_area'}) or \
                         soup.find('div', class_='viewer') or \
                         soup.find('div', class_='comic_viewer')
        
        if not viewer_area:
            print("웹툰 이미지 영역을 찾을 수 없습니다.")
            return
        
        # 이미지 태그들 찾기
        img_tags = viewer_area.find_all('img')
        
        if not img_tags:
            print("웹툰 이미지를 찾을 수 없습니다.")
            return
        
        print(f"총 {len(img_tags)}개의 이미지를 발견했습니다.")
        
        downloaded_count = 0
        
        for i, img_tag in enumerate(img_tags, 1):
            try:
                img_src = img_tag.get('src')
                if not img_src:
                    continue
                
                # 이미지 URL이 상대 경로인 경우 절대 경로로 변환
                img_url = urljoin('https://comic.naver.com', img_src)
                
                # 파일명 생성 (확장자 추출)
                file_extension = os.path.splitext(img_url.split('?')[0])[-1]
                if not file_extension:
                    file_extension = '.jpg'  # 기본 확장자
                
                filename = f"{i:03d}{file_extension}"  # 001.jpg, 002.jpg 형식
                filepath = os.path.join(webtoon_dir, filename)
                
                # 이미지 다운로드를 위한 헤더 (Referer 중요)
                img_headers = {
                    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
                    'Referer': url
                }
                
                print(f"이미지 다운로드 중... [{i}/{len(img_tags)}] {filename}")
                
                img_response = requests.get(img_url, headers=img_headers, stream=True)
                img_response.raise_for_status()
                
                # 이미지 파일 저장
                with open(filepath, 'wb') as f:
                    for chunk in img_response.iter_content(chunk_size=8192):
                        if chunk:
                            f.write(chunk)
                
                downloaded_count += 1
                print(f"✓ 저장 완료: {filepath}")
                
                # 요청 간격 조정 (서버 부하 방지)
                time.sleep(0.5)
                
            except Exception as e:
                print(f"이미지 다운로드 실패 [{i}]: {e}")
                continue
        
        print(f"\n다운로드 완료!")
        print(f"성공: {downloaded_count}/{len(img_tags)}개 이미지")
        print(f"저장 위치: {webtoon_dir}")
        
    except requests.RequestException as e:
        print(f"웹페이지 접근 실패: {e}")
    except Exception as e:
        print(f"예상치 못한 오류 발생: {e}")

def download_webtoon_with_retry(title, no, url, max_retries=3):
    """
    재시도 기능이 포함된 웹툰 다운로드 함수
    """
    for attempt in range(max_retries):
        try:
            print(f"\n시도 {attempt + 1}/{max_retries}")
            download_one_episode(title, no, url)
            return  # 성공하면 함수 종료
        except Exception as e:
            print(f"시도 {attempt + 1} 실패: {e}")
            if attempt < max_retries - 1:
                print("5초 후 재시도합니다...")
                time.sleep(5)
            else:
                print("모든 시도가 실패했습니다.")

# 사용 예제
if __name__ == "__main__":
    # 예제 실행
    title = '일렉시드'
    episode_no = 341
    webtoon_url = 'https://comic.naver.com/webtoon/detail?titleId=717481&no=341&week=wed'
    
    print("=" * 60)
    print(f"웹툰 다운로드 시작")
    print(f"제목: {title}")
    print(f"회차: {episode_no}")
    print(f"URL: {webtoon_url}")
    print("=" * 60)
    
    # 기본 다운로드
    download_one_episode(title, episode_no, webtoon_url)
    
    # 또는 재시도 기능 포함된 버전 사용
    # download_webtoon_with_retry(title, episode_no, webtoon_url)

웹툰 다운로드 시작
제목: 일렉시드
회차: 341
URL: https://comic.naver.com/webtoon/detail?titleId=717481&no=341&week=wed
디렉토리 생성: img\일렉시드\341
웹툰 페이지 접근 중: https://comic.naver.com/webtoon/detail?titleId=717481&no=341&week=wed
총 88개의 이미지를 발견했습니다.
이미지 다운로드 중... [1/88] 001.jpg
✓ 저장 완료: img\일렉시드\341\001.jpg
이미지 다운로드 중... [2/88] 002.jpg
✓ 저장 완료: img\일렉시드\341\002.jpg
이미지 다운로드 중... [3/88] 003.jpg
✓ 저장 완료: img\일렉시드\341\003.jpg
이미지 다운로드 중... [4/88] 004.jpg
✓ 저장 완료: img\일렉시드\341\004.jpg
이미지 다운로드 중... [5/88] 005.jpg
✓ 저장 완료: img\일렉시드\341\005.jpg
이미지 다운로드 중... [6/88] 006.jpg
✓ 저장 완료: img\일렉시드\341\006.jpg
이미지 다운로드 중... [7/88] 007.jpg
✓ 저장 완료: img\일렉시드\341\007.jpg
이미지 다운로드 중... [8/88] 008.jpg
✓ 저장 완료: img\일렉시드\341\008.jpg
이미지 다운로드 중... [9/88] 009.jpg
✓ 저장 완료: img\일렉시드\341\009.jpg
이미지 다운로드 중... [10/88] 010.jpg
✓ 저장 완료: img\일렉시드\341\010.jpg
이미지 다운로드 중... [11/88] 011.jpg
✓ 저장 완료: img\일렉시드\341\011.jpg
이미지 다운로드 중... [12/88] 012.jpg
✓ 저장 완료: img\일렉시드\341\012.jpg
이미지 다운로드 중... [13/88] 013.jpg
✓ 저장 완료: img\일렉시드\341\013.jpg
이